# 환율 라벨링 후 ngram과 merge 하기

In [1]:
import pandas as pd

In [18]:
news = pd.read_csv('C:/Users/user/Desktop/globo/news_ngram.csv')

In [19]:
news

,date,content,main_title,sub_title,total,unigram,bigram,trigram,fourgram,fivegram
0,2018-06-01,Em um último movimento para tentar ressuscitar...,Áudios convocam caminhoneiros para novo ato,"Em vídeos, um dos líderes do movimento pede qu...",Áudios convocam caminhoneiros para novo atoEm ...,"áudi,convoc,caminhoneir,nov,ato,víd,líd,movime...","áudi;convoc,convoc;caminhoneir,caminhoneir;nov...","áudi;convoc;caminhoneir,convoc;caminhoneir;nov...","áudi;convoc;caminhoneir;nov,convoc;caminhoneir...","áudi;convoc;caminhoneir;nov;ato,convoc;caminho..."
1,2018-06-01,As ações da Petrobras fecharam em queda de qua...,"Ações da Petrobras caem quase 15%, e estatal p...",Recibos de ações negociadas nos Estados Unidos...,"Ações da Petrobras caem quase 15%, e estatal p...","açõ,petrobr,caem,quas,15,estatal,perd,r,40,bi,...","açõ;petrobr,petrobr;caem,caem;quas,quas;15,15;...","açõ;petrobr;caem,petrobr;caem;quas,caem;quas;1...","açõ;petrobr;caem;quas,petrobr;caem;quas;15,cae...","açõ;petrobr;caem;quas;15,petrobr;caem;quas;15;..."
2,2018-06-01,O Ministério do Trabalho suspendeu por 30 dias...,"Após operação da PF, ministério suspende regis...",Decisão paralisa análises dos registros. Deput...,"Após operação da PF, ministério suspende regis...","após,oper,pf,ministéri,suspend,registr,sindic,...","após;oper,oper;pf,pf;ministéri,ministéri;suspe...","após;oper;pf,oper;pf;ministéri,pf;ministéri;su...","após;oper;pf;ministéri,oper;pf;ministéri;suspe...","após;oper;pf;ministéri;suspend,oper;pf;ministé..."
3,2018-06-01,"Depois de , pode ser o da , gigante de alime...","Após pedir demissão, Parente pode ser o novo p...",Parente assumiu recentemente o comando do cons...,"Após pedir demissão, Parente pode ser o novo p...","após,ped,demissã,parent,pod,ser,nov,presidente...","após;ped,ped;demissã,demissã;parent,parent;pod...","após;ped;demissã,ped;demissã;parent,demissã;pa...","após;ped;demissã;parent,ped;demissã;parent;pod...","após;ped;demissã;parent;pod,ped;demissã;parent..."
4,2018-06-01,O conselho de administração da Petrobras indic...,Petrobras confirma Ivan Monteiro como novo pre...,"Monteiro, atual diretor financeiro, está a cam...",Petrobras confirma Ivan Monteiro como novo pre...,"petrobr,confirm,ivan,monteir,nov,president,int...","petrobr;confirm,confirm;ivan,ivan;monteir,mont...","petrobr;confirm;ivan,confirm;ivan;monteir,ivan...","petrobr;confirm;ivan;monteir,confirm;ivan;mont...","petrobr;confirm;ivan;monteir;nov,confirm;ivan;..."
...,...,...,...,...,...,...,...,...,...,...
9527,2019-12-31,"O presidente dos Estados Unidos, Donald Trump,...",Trump anuncia que assinará o acordo comercial ...,Presidente dos Estados Unidos disse que irá ao...,Trump anuncia que assinará o acordo comercial ...,"trump,anunc,assin,acord,comercial,parcial,chin...","trump;anunc,anunc;assin,assin;acord,acord;come...","trump;anunc;assin,anunc;assin;acord,assin;acor...","trump;anunc;assin;acord,anunc;assin;acord;come...","trump;anunc;assin;acord;comercial,anunc;assin;..."
9528,2019-12-31,O governo informou que transferiu nesta terça-...,"Governo transfere R$ 11,7 bilhões a estados e ...",Equipe econômica espera avançar com o pacto fe...,"Governo transfere R$ 11,7 bilhões a estados e ...","govern,transfer,r,11,7,bilhõ,estad,municípi,;,...","govern;transfer,transfer;r,r;11,7,11,7;bilhõ,b...","govern;transfer;r,transfer;r;11,7,r;11,7;bilhõ...","govern;transfer;r;11,7,transfer;r;11,7;bilhõ,r...","govern;transfer;r;11,7;bilhõ,transfer;r;11,7;b..."
9529,2019-12-31,"O presidente dos Estados Unidos, , disse nesta...",Trump diz que acordo comercial EUA-China será ...,Fase 1 do tratado será celebrada na Casa Branc...,Trump diz que acordo comercial EUA-China será ...,"trump,diz,acord,comercial,eua-chin,assin,15,ja...","trump;diz,diz;acord,acord;comercial,comercial;...","trump;diz;acord,diz;acord;comercial,acord;come...","trump;diz;acord;comercial,diz;acord;comercial;...","trump;diz;acord;comercial;eua-chin,diz;acord;c..."
9530,2019-12-31,"O presidente do Supremo Tribunal Federal, mini

### 환율 라벨링

In [4]:
currency = pd.read_csv('C:/Users/user/Desktop/환율.csv', encoding='euc-kr')
currency

,날짜,종가,오픈,고가,저가,변동 %
0,2020년 01월 01일,287.35,287.37,287.49,287.16,-0.01%
1,2019년 12월 31일,287.37,288.10,288.40,286.90,-0.25%
2,2019년 12월 30일,288.09,287.06,288.79,285.55,0.51%
3,2019년 12월 27일,286.63,286.53,287.45,285.50,0.02%
4,2019년 12월 26일,286.56,284.29,287.25,284.12,0.82%
...,...,...,...,...,...,...
409,2018년 06월 07일,274.38,277.14,278.35,270.64,-1.03%
410,2018년 06월 06일,277.24,281.50,281.73,276.66,-1.54%
411,2018년 06월 05일,281.58,285.79,286.50,280.73,-1.45%
412,2018년 06월 04일,285.71,284.51,287.35,283.65,0.32%


날짜 형태에 맞게 변환

In [5]:
currency['날짜'] = currency['날짜'].str.replace('년 ','-')
currency['날짜'] = currency['날짜'].str.replace('월 ','-')
currency['날짜'] = currency['날짜'].str.replace('일','')
currency['날짜']=pd.to_datetime(currency['날짜'],format="%Y-%m-%d")

In [6]:
ls = []
for i in currency['변동 %']:
    if len(i) == 5:
        ls.append('+'+i)
    else:
        ls.append(i)

In [7]:
a = pd.DataFrame(ls, columns=['변동'])
real = pd.concat([currency, a], axis=1)

In [8]:
real['변동'] = real['변동'].str[:5]
real = real.drop(['종가', '오픈', '고가','저가', '변동 %'], axis=1)
real['변동']=pd.to_numeric(real['변동'])

In [9]:
# 라벨링 칼럼 추가
real['label']=''

In [10]:
for i in range(len(real)):
    if real['변동'][i] >= 0.4:
        real['label'][i] = real['label'][i].replace('', '1')
    elif real['변동'][i] <= -0.4:
        real['label'][i] = real['label'][i].replace('', '-1')
    else:
        real['label'][i] = real['label'][i].replace('', '0')


<ipython-input-10-de7073b1d3b4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['label'][i] = real['label'][i].replace('', '0')
<ipython-input-10-de7073b1d3b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['label'][i] = real['label'][i].replace('', '1')
<ipython-input-10-de7073b1d3b4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['label'][i] = real['label'][i].replace('', '-1')


### 뉴스와 merge

In [134]:
real.to_csv('C:/Users/user/Desktop/globo/real_krw.csv', index=False)

In [20]:
real = pd.read_csv('C:/Users/user/Desktop/globo/real_krw.csv')

In [21]:
real.rename(columns = {'날짜' : 'date'}, inplace = True)

In [22]:
news_label = pd.merge(news,real,on='date',how='inner')
news_label.drop(['content', 'main_title', 'sub_title', 'total', '변동'], axis='columns', inplace=True)

In [23]:
news_label

,date,unigram,bigram,trigram,fourgram,fivegram,label
0,2018-06-01,"áudi,convoc,caminhoneir,nov,ato,víd,líd,movime...","áudi;convoc,convoc;caminhoneir,caminhoneir;nov...","áudi;convoc;caminhoneir,convoc;caminhoneir;nov...","áudi;convoc;caminhoneir;nov,convoc;caminhoneir...","áudi;convoc;caminhoneir;nov;ato,convoc;caminho...",-1
1,2018-06-01,"açõ,petrobr,caem,quas,15,estatal,perd,r,40,bi,...","açõ;petrobr,petrobr;caem,caem;quas,quas;15,15;...","açõ;petrobr;caem,petrobr;caem;quas,caem;quas;1...","açõ;petrobr;caem;quas,petrobr;caem;quas;15,cae...","açõ;petrobr;caem;quas;15,petrobr;caem;quas;15;...",-1
2,2018-06-01,"após,oper,pf,ministéri,suspend,registr,sindic,...","após;oper,oper;pf,pf;ministéri,ministéri;suspe...","após;oper;pf,oper;pf;ministéri,pf;ministéri;su...","após;oper;pf;ministéri,oper;pf;ministéri;suspe...","após;oper;pf;ministéri;suspend,oper;pf;ministé...",-1
3,2018-06-01,"após,ped,demissã,parent,pod,ser,nov,presidente...","após;ped,ped;demissã,demissã;parent,parent;pod...","após;ped;demissã,ped;demissã;parent,demissã;pa...","após;ped;demissã;parent,ped;demissã;parent;pod...","após;ped;demissã;parent;pod,ped;demissã;parent...",-1
4,2018-06-01,"petrobr,confirm,ivan,monteir,nov,president,int...","petrobr;confirm,confirm;ivan,ivan;monteir,mont...","petrobr;confirm;ivan,confirm;ivan;monteir,ivan...","petrobr;confirm;ivan;monteir,confirm;ivan;mont...","petrobr;confirm;ivan;monteir;nov,confirm;ivan;...",-1
...,...,...,...,...,...,...,...
8524,2019-12-31,"trump,anunc,assin,acord,comercial,parcial,chin...","trump;anunc,anunc;assin,assin;acord,acord;come...","trump;anunc;assin,anunc;assin;acord,assin;acor...","trump;anunc;assin;acord,anunc;assin;acord;come...","trump;anunc;assin;acord;comercial,anunc;assin;...",0
8525,2019-12-31,"govern,transfer,r,11,7,bilhõ,estad,municípi,;,...","govern;transfer,transfer;r,r;11,7,11,7;bilhõ,b...","govern;transfer;r,transfer;r;11,7,r;11,7;bilhõ...","govern;transfer;r;11,7,transfer;r;11,7;bilhõ,r...","govern;transfer;r;11,7;bilhõ,transfer;r;11,7;b...",0
8526,2019-12-31,"trump,diz,acord,comercial,eua-chin,assin,15,ja...","trump;diz,diz;acord,acord;comercial,comercial;...","trump;diz;acord,diz;acord;comercial,acord;come...","trump;diz;acord;comercial,diz;acord;comercial;...","trump;diz;acord;comercial;eua-chin,diz;acord;c...",0
8527,2019-12-31,"toffol,suspend,resolu,reduz,valor,dpvat,2020pa...","toffol;suspend,suspend;resolu,resolu;reduz,red...","toffol;suspend;resolu,suspend;resolu;reduz,res...","toffol;suspend;resolu;reduz,suspend;resolu;red...","toffol;suspend;resolu;reduz;valor,suspend;reso...",0


In [24]:
news_label['ngram']=news_label['unigram']+news_label['bigram']+news_label['trigram']+news_label['fourgram']+news_label['fivegram']
df = news_label.drop(['unigram', 'bigram','trigram', 'fourgram', 'fivegram'], axis=1)
df

,date,label,ngram
0,2018-06-01,-1,"áudi,convoc,caminhoneir,nov,ato,víd,líd,movime..."
1,2018-06-01,-1,"açõ,petrobr,caem,quas,15,estatal,perd,r,40,bi,..."
2,2018-06-01,-1,"após,oper,pf,ministéri,suspend,registr,sindic,..."
3,2018-06-01,-1,"após,ped,demissã,parent,pod,ser,nov,presidente..."
4,2018-06-01,-1,"petrobr,confirm,ivan,monteir,nov,president,int..."
...,...,...,...
8524,2019-12-31,0,"trump,anunc,assin,acord,comercial,parcial,chin..."
8525,2019-12-31,0,"govern,transfer,r,11,7,bilhõ,estad,municípi,;,..."
8526,2019-12-31,0,"trump,diz,acord,comercial,eua-chin,assin,15,ja..."
8527,2019-12-31,0,"toffol,suspend,resolu,reduz,valor,dpvat,2020pa..."


In [25]:
df.to_csv('C:/Users/user/Desktop/globo/ngram_label.csv', index=False)

In [26]:
df['ngram'][0]

'áudi,convoc,caminhoneir,nov,ato,víd,líd,moviment,ped,motor,vá,estádi,man,garrincha,últim,moviment,tent,ressuscit,paralis,caminhoneir,grup,lider,motor,wallac,landim,chorã,catalã,go,quer,reun,manifest,brasíl,doming,avali,condiçõ,nov,paralisação.,víd,replic,aplic,celul,chorã,ped,motor,desloqu,estádi,man,garrinch,capital,federal,a,concentr,vai,ser,doming,estádi,man,garrinch,segunda-feir,gent,consegu,mont,comissã,represent,cad,estad,gent,pleit,convers,govern,diss,chorã,víd,grav,nest,sexta-feira.,segunda-feir,convers,govern,tod,categor,vam,decid,vai,ser,se,tod,categor,decid,segunda-feir,zer,hor,terc,gent,paralis,gent,vai,paralis,novo.,quand,brasíl,seman,pass,chorã,consegu,mant,audiênc,integr,paláci,planalt,dess,form,procur,reconhec,formal,lider,esvazi,papel,paralisação.,lider,chorã,pass,ser,question,vári,caminhoneir,ped,víd,quarta-feir,30,fim,protest,bloquei,estradas.,caminhoneir,edson,alves,silv,40,cheg,estádi,tard,quinta-feir,ele,particip,vári,grup,aplic,celul,diss,ter,1,5,mil,coleg,camin